### Retriever

In [ ]:
from langchain_community.vectorstores import FAISS as VectorStore
from langchain_community.embeddings import GPT4AllEmbeddings

embeddings = GPT4AllEmbeddings()
store = VectorStore.load_local("../../retrieve/vector_store", embeddings, allow_dangerous_deserialization=True)
retriever = store.as_retriever(search_kwargs={"k": 4})

### Model

In [ ]:
# import dotenv
# dotenv.load_dotenv()

In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id= "mistralai/Mistral-7B-Instruct-V0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)
llm = HuggingFacePipeline(pipeline=pipe)

### Chains
https://python.langchain.com/docs/modules/memory/adding_memory_chain_multiple_inputs/

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate

template = """You are a chatbot having a conversation with a human.

Given the following extracted parts of a long document and a question, create a final answer.

{context}

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")

chain = load_qa_chain(llm=llm, chain_type="stuff", memory=memory, prompt=prompt)

In [ ]:
%time
query = "How install the chatbots"
docs = retriever.invoke(query)

chain.invoke({"input_documents": docs, "human_input": query}) #, return_only_outputs=True)